<a href="https://colab.research.google.com/github/ssanchezgoe/curso_deep_learning_economia/blob/main/NBs_Google_Colab/DL_S05_Regresion_Lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="140px" src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" align="left" hspace="10px" vspace="0px"></p>

<h1> Curso Deep Learning: Economía</h1>

## S04: Regresión Lineal

## Regresión Lineal

El problema de regresión lineal en una dimensión tiene como objetivo ajustar una variable numérica y continua,$y$, a una variable independiente $x$, por medio de un modelo lineal

$$y = mx + b$$

en donde los parámetros corresponden a:

- $m$ pendiente de la recta.
- $b$ intercepto con el eje $y$.

In [ ]:
#@title Librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from numpy.random import randn
from numpy.random import seed

In [ ]:
x=10*np.random.rand(100)
e=10*(np.random.rand(100)-0.5)
y=3*x+5+e
plt.scatter(x,y)
plt.plot(np.arange(0,11),3*np.arange(0,11)+5,c='r')
plt.show()

La correlación entre ambos datos esta dada por:

In [ ]:
# Cálculo de la correlación de pearson.
corr, _ = pearsonr(x, y)
corr

La línea recta sobre los puntos representa la tendencia del ajuste. En la figura se puede observar que esta línea no se ajusta a cada uno de los puntos; en su lugar, marca una tendencia promedio, por lo que se puede obtener un error en el ajuste a través de la distancia promedio de los puntos a la línea de tendencia:

In [ ]:
plt.errorbar(x,y,yerr=np.c_[e,np.zeros_like(e)].T,fmt='none',c='m')
plt.scatter(x,y)
plt.plot(np.arange(0,11),3*np.arange(0,11)+5,c='r')
plt.show()

### Mínimos Cuadrados: Solución Analítica

Al rededor de 1800 Carl Friederich Gauss y Adrien-Marie Legendre encontraron la forma de realizar un ajuste lineal, al minimizar la suma de los errores cuadráticos (MSE):

$$\text{MSE} = \frac{1}{N}\sum_{i=1}^N(\hat{y}_i-y_i)^2$$

en donde 

- $\hat{y}_i$ corresponde al i-ésimo valor predicho por el modelo.
- $y_i$ corresponde al i-ésimo valor del conjunto de datos de la variable objetivo.

Se puede mostrar que los valores de la pendiente $m$ y el intercepto $c$ que minimizan la ecuación $y = mx + b$ están dados por

$$m=\frac{\bar{xy}-\bar{x}\bar{y}}{\bar{x^2}-\bar{x}^2}$$

$$b=\bar{y}-m\bar{x}$$

en donde las cantidades con una barra corresponden a sus valores promedio.

Los valores optimos serán entonces

In [ ]:
N=len(x)
m=(np.sum(x*y)-np.sum(x)*np.sum(y)/N)/(np.sum(x*x)-np.sum(x)**2/N)
b=(np.sum(y)-m*np.sum(x))/N

print('m',m)
print('b',b)

In [ ]:
plt.scatter(x,y)
plt.plot(np.arange(0,10,10/100), m*np.arange(0,10,10/100)+b, c='r')
plt.legend(["Modelo","Datos"])
plt.show()

In [ ]:
 print(f"MSE: {mean_squared_error(y, m*x+b)}")

El problema multidimensional de un ajuste lineal puede ser escrito como:

$$y=a_0+a_1 x_1+a_2 x_2+...+a_m x_m=\sum_{j=0}^m a_m x_m$$

El cual puede ser resuelto analíticamente como:

$$\vec{a}=(a_0,a_1,...,a_m)=(X^T X)^{-1}X^T y$$

Para lo que ser requiere de la inversión de la matriz $X^T X$

In [ ]:
X=np.c_[np.ones_like(x),x]
params = np.matmul(np.linalg.inv(np.matmul(X.T,X)),np.matmul(X.T,y))
params

In [ ]:
plt.scatter(x,y)
plt.plot(np.arange(0,10,10/100), params[1]*np.arange(0,10,10/100)+params[0], c='r')
plt.legend(["Modelo","Datos"])
plt.show()

In [ ]:
print(f"MSE: {mean_squared_error(y, params[1]*x+params[0])}")

No obstante, si se poseen variabes características, la ecuación normal puede resultar computacionalmente lenta, debido a la inversión matricial que se reaquiere calcular. Para evitar esto, se hace uso de la técnica de descenso del gradiente.

##  Solución en `sklearn`:

La solución estándar en `sklearn` consiste en la siguientes líneas:

In [ ]:
lr = linear_model.LinearRegression()
lr.fit(x.reshape(-1,1),y)
print(f"{lr.intercept_, lr.coef_[0]}")

In [ ]:
plt.scatter(x,y)
plt.plot(np.arange(0,10,10/100), lr.coef_[0]*np.arange(0,10,10/100)+lr.intercept_, c='r')
plt.legend(["Modelo","Datos"])
plt.show()

In [ ]:
print(f"MSE: {mean_squared_error(y, lr.coef_[0]*x+lr.intercept_)}")

Si usamos decenso de gradiente, usamos el objeto de `sklearn` `SGDRegresor`:

In [ ]:
model = SGDRegressor(max_iter=1000, tol=0.001, eta0=0.1)
model.fit(x.reshape(-1,1),y)
print(f"{model.intercept_, model.coef_[0]}")

In [ ]:
plt.scatter(x,y)
plt.plot(np.arange(0,10,10/100), model.coef_[0]*np.arange(0,10,10/100)+model.intercept_, c='r')
plt.legend(["Modelo","Datos"])
plt.show()

In [ ]:
print(f"MSE: {mean_squared_error(y, model.coef_[0]*x+model.intercept_)}")

## Solución en `keras`:

En `keras`, para implementar una regresión lineal, se programa una sola capa sin función de activación:

In [ ]:
from tensorflow import keras

In [ ]:
keras.backend.clear_session()
model = keras.Sequential()
model.add(keras.layers.Dense(1, input_shape=(1,)))
model.summary()

In [ ]:
model.compile(keras.optimizers.Adam(learning_rate=0.9), 'mean_squared_error')

In [ ]:
model.fit(x,y,epochs=30)

In [ ]:
m,b = model.get_weights()
print(f"{b[0],m[0][0]}")

In [ ]:
plt.scatter(x,y)
plt.plot(np.arange(0,10,10/100), m[0][0]*np.arange(0,10,10/100)+b[0], c='r')
plt.legend(["Modelo","Datos"])
plt.show()

In [ ]:
print(f"MSE: {mean_squared_error(y, m[0][0]*x+b[0])}")

## Ejercicios

Para el siguiente conjunto de puntos

In [ ]:
#@title Conjunto de datos xE, yE
from sklearn.datasets import make_regression

xE,yE = make_regression(n_samples=200, n_features=1, noise=3)
plt.scatter(xE,yE)
plt.xlabel("xE")
plt.ylabel("yE")
plt.show()

realice los ejercicios enumerados a continuación.

### Ejercicio 1:

Para este ejercicio: 
1. Realice un ajuste lineal, implementando una función para el método de mínimos cuadrados
2. Desarrolle una función que grafique el conjunto de datos y el modelo. 
3. Calcule el valor del error cuadrático medio.

### Ejercicio 2:

Para este ejercicio:
1. Realice un ajuste lineal siguiendo el método estándard de `skelearn`. 
2. Haga uso de la función implementada en el ejercicio 1 para visualizar los datos y el modelo.
3. Calcule el valor del error cuadrático medio.

### Ejercicio 3:

Para este ejercicio:
1. Realice un ajuste lineal siguiendo el método de descenso de gradiente estocástico implementado en `sklearn`, usando varios modelos en donde evalue varios valores de los parámetros vistos:

  - `max_iter`
  - `tol`
  - `eta0`

2. Visualice los datos junto al modelo, en cada conjunto de parámetros analizado.
3. Calcule el valor del error cuadrático medio.

### Ejercicio 4:

Para este ejercicio:
1. Realice un ajuste lineal implementando el método de `keras`. 
2. Visualice el modelo junto a los datos.
3. Calcule el valor del error cuadrático medio.